In [11]:
from nltk import sent_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from tqdm.notebook import tqdm
from torchmetrics.text import ROUGEScore

In [3]:
df = pd.read_csv("../../../data/labelled/metadata/splits/test.csv")

In [12]:
rouge = ROUGEScore()

In [4]:
df.head(2)

,product_info,summary
0,<b>CONTENTS - In the Friendly Swede Retail pa...,The Friendly Swede Retail packaging includes a...
1,EMPIRE's two piece snap-on cases are made of h...,EMPIRE's two piece snap-on cases provide maxim...


In [5]:
vectorizer = TfidfVectorizer(stop_words=stopwords.words("english"))

In [13]:
preds = []
rouge_scores = {"rouge1":[], "rouge2":[], "rougeL": []}
for idx in tqdm(range(len(df))):
    sentences = sent_tokenize(df.product_info[idx])
    doc_term_matrix = vectorizer.fit_transform(sentences)
    scores = doc_term_matrix.sum(1).flatten().tolist()[0]
    sorted_pairs = sorted(zip(sentences,scores), key=lambda x: x[1], reverse=True)
    max_len = len(df.product_info[idx])//2
    cur_len, idx = 0, 0
    predicted_summary = ""
    while cur_len<=max_len:
        predicted_summary+=f"{sorted_pairs[idx][0]} "
        cur_len+=len(sorted_pairs[idx][0])
        idx+=1
    preds.append(predicted_summary.strip())
    rouge_all = rouge(predicted_summary, df.summary[idx])
    rouge_scores['rouge1'].append(rouge_all['rouge1_fmeasure'].item())
    rouge_scores['rouge2'].append(rouge_all['rouge2_fmeasure'].item())
    rouge_scores['rougeL'].append(rouge_all['rougeL_fmeasure'].item())

  0%|          | 0/25086 [00:00<?, ?it/s]

In [14]:
df['rouge1'] = rouge_scores['rouge1']
df['rouge2'] = rouge_scores['rouge2']
df['rougeL'] = rouge_scores['rougeL']
df['tfidf'] = preds

In [15]:
df.to_csv("tfidf_descriptions_scores.csv")

In [16]:
scores = {"rouge1": df.rouge1.mean(), "rouge2": df.rouge2.mean(), "rougeL": df.rougeL.mean()}

In [17]:
scores

{'rouge1': 0.11878613434333431,
 'rouge2': 0.008587316862061775,
 'rougeL': 0.08028587055828937}